## Demo of fmm Python API

In [1]:
from fmm import Network,NetworkGraph,FastMapMatch,FastMapMatchConfig,UBODT

### Load network data and graph

In [2]:
import os
PWD = os.getcwd() # os.path.abspath(os.path.dirname(__file__))
network = Network(f"{PWD}/../data/edges.shp")
print("Nodes {} edges {}".format(network.get_node_count(),network.get_edge_count()))
graph = NetworkGraph(network)

Nodes 17 edges 30
[2023-06-04 11:09:09.719] [info] [network.cpp:155] Read network from file /home/tzx/git/fmm/example/notebook/../data/edges.shp
[2023-06-04 11:09:09.751] [info] [network.cpp:232] Number of edges 30 nodes 17
[2023-06-04 11:09:09.751] [info] [network.cpp:233] Field index: id 1 source 2 target 3
[2023-06-04 11:09:09.751] [info] [network.cpp:236] Read network done.
[2023-06-04 11:09:09.752] [info] [network_graph.cpp:18] Construct graph from network edges start
[2023-06-04 11:09:09.752] [info] [network_graph.cpp:31] Graph nodes 17 edges 30
[2023-06-04 11:09:09.752] [info] [network_graph.cpp:32] Construct graph from network edges end


In [3]:
dumps = network.dumps()
print(dumps[:500])
print(len(dumps))

{"srid":4326,"edges":[{"id":1,"source":1,"target":2,"coordinates":[[2.0,1.0],[2.0,0.0]]},{"id":2,"source":2,"target":1,"coordinates":[[2.0,0.0],[2.0,1.0]]},{"id":3,"source":3,"target":1,"coordinates":[[3.0,1.0],[2.0,1.0]]},{"id":4,"source":4,"target":3,"coordinates":[[4.0,1.0],[3.0,1.0]]},{"id":5,"source":1,"target":5,"coordinates":[[2.0,1.0],[2.0,2.0]]},{"id":6,"source":5,"target":1,"coordinates":[[2.0,2.0],[2.0,1.0]]},{"id":7,"source":3,"target":6,"coordinates":[[3.0,1.0],[3.0,2.0]]},{"id":8,"
2097


In [1]:
# from/to_json
nn = Network()
nn.loads(dumps)
redumps = nn.dumps()
assert redumps == dumps, f'old: {dumps},\nnew: {redumps}'
# network.dump('network.json')

NameError: name 'Network' is not defined

### Precompute an UBODT file

**This step can be skipped if you already created one UBODT file.**

In [6]:
from fmm import UBODTGenAlgorithm

In [7]:
ubodt_gen = UBODTGenAlgorithm(network,graph)

In [8]:
status = ubodt_gen.generate_ubodt(f"{PWD}/../data/ubodt.txt", 4, binary=False, use_omp=True)
print(status)

[2023-06-04 10:36:16.393] [info] [ubodt_gen_algorithm.cpp:94] Start to generate UBODT with delta 4
[2023-06-04 10:36:16.393] [info] [ubodt_gen_algorithm.cpp:95] Output format csv
Status: success
Time takes 0.024 seconds

[2023-06-04 10:36:16.405] [info] [ubodt_gen_algorithm.cpp:124] Progress 10 / 17


In [ ]:
path = f"{PWD}/../data/ubodt.json"
status = ubodt_gen.generate_ubodt(path, 4, binary=False)
print(status)

### Load UBODT data

In [ ]:
ubodt = UBODT.read_ubodt_csv(f"{PWD}/../data/ubodt.txt")

### Create FMM model

In [ ]:
model = FastMapMatch(network,graph,ubodt)

### Define FMM configuration

In [ ]:
k = 4
radius = 0.4
gps_error = 0.5
fmm_config = FastMapMatchConfig(k,radius,gps_error)

### Run map matching

#### Match a single trajectory

In [ ]:
wkt = "LINESTRING(0.200812146892656 2.14088983050848,1.44262005649717 2.14879943502825,3.06408898305084 2.16066384180791,3.06408898305084 2.7103813559322,3.70872175141242 2.97930790960452,4.11606638418078 2.62337570621469)"

In [ ]:
result = model.match_wkt(wkt,fmm_config)

In [ ]:
print("Matched path: ", list(result.cpath))
print("Matched edge for each point: ", list(result.opath))
print("Matched edge index ",list(result.indices))
print("Matched geometry: ",result.mgeom.export_wkt())
print("Matched point ", result.pgeom.export_wkt())

##### Print detailed match information

In [ ]:
candidates = []
for c in result.candidates:
    candidates.append((c.edge_id,c.source,c.target,c.error,c.length,c.offset,c.spdist,c.ep,c.tp))

In [ ]:
import pandas as pd
df = pd.DataFrame(candidates,
                  columns=["eid","source","target","error","length","offset","spdist","ep","tp"])

In [ ]:
df.head()

#### Match trajectories in a GPS file

In [ ]:
from fmm import GPSConfig,ResultConfig

In [ ]:
input_config = GPSConfig()
input_config.file = f"{PWD}/../data/trips.csv"
input_config.id = "id"

In [ ]:
print(input_config.to_string())

In [ ]:
result_config = ResultConfig()
result_config.file = f"{PWD}/../data/mr.txt"
result_config.output_config.write_opath = True
print(result_config.to_string())

In [ ]:
status = model.match_gps_file(input_config, result_config, fmm_config)

In [ ]:
print(status)